### Authentication by encrypted negative password

## Authentication

In [2]:
import os
import binascii
import json
import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES
import random

##### converting the given password into bytes

In [3]:
password = "pavaniMandadi8"
byte_pass = bytes(password, "utf-8")
byte_pass

b'pavaniMandadi8'

##### Generating the Hex digit and binary digit of the given byte string

In [54]:
hash_256 = hashlib.sha256()
hash_256.update(byte_pass)
hex_digit = hash_256.hexdigest()
binary_digit = bin(int(hex_digit, 16))[2:].zfill(256)   #"{0:08b}".format(int(hex_digit, 16))
print("Hex Digit of given password :",hex_digit)
print("Binary Digit of given password :",binary_digit)

Hex Digit of given password : c290f15db1f4c02480186dc96d2a35622e04e116520f3502023a8affa9738d6d
Binary Digit of given password : 1100001010010000111100010101110110110001111101001100000000100100100000000001100001101101110010010110110100101010001101010110001000101110000001001110000100010110010100100000111100110101000000100000001000111010100010101111111110101001011100111000110101101101


##### FInding the Nth permutation of the binary digit - N is the last permutation of the given password string

In [20]:
def npermute(binary_digit):
    return binary_digit[::-1]

n_permutated_str = npermute(binary_digit) 

print("The % Permutation of binary digit is : %s", len(password), n_permutated_str)

The % Permutation of binary digit is : %s 14 1011011010110001110011101001010111111111010100010101110001000000010000001010110011110000010010100110100010000111001000000111010001000110101011000101010010110110100100111011011000011000000000010010010000000011001011111000110110111010100011110000100101000011


##### Generating negative database with sequence of symbols

In [21]:
symbols = {"00": "0", "01": "1", "11": "*", "10": "*"}
sequence_of_symbols = "".join([symbols[n_permutated_str[each: each + 2]] for each in range(0, len(n_permutated_str)) if each < len(n_permutated_str)-1])

##### Negate the permutated string if the ndb ends with * and generate ndb with negated string

In [22]:
def negate(symbols_string):
    return "".join([str(int(not(int(each)))) for each in symbols_string])

n_permutated_negate = negate(n_permutated_str)
sequence_of_symbols_2 = "".join([symbols[n_permutated_negate[each: each + 2]] for each in range(0, len(n_permutated_negate)) if each < len(n_permutated_negate)-1])
sequence_of_symbols_2

'1*01*01*1*01***001**001*1**1*1*000000001*1*1***1*1*001***1*******1******1*1*01**0001*****1**1*1**01*1***1****001**1******001*1***1***01*1*1*01***1*1*1**1*01*01*1**1**001*01*01****01**********1**1**1********01**1*00001***01*01*001*1*1***0001****1**1*1****0'

##### Encrypting Negative database using AES

In [23]:
BLOCK_SIZE = 16
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)


def encrypt(raw, password):
    private_key = hashlib.sha256(password.encode("utf-8")).digest()
    raw = pad(raw)
    iv = Random.new().read(AES.block_size)
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return base64.b64encode(iv + cipher.encrypt(raw))

encoded_string = encrypt(sequence_of_symbols_2, password)

print("AES Encrypted String :", encoded_string)

AES Encrypted String : b'VzxW7nuvwYDrTBMktMNNxnTc9I/2A0yzA3/oE1V8ZjAuKJA8cXSDPMgLslFKaV/1RTRdFUM+BQzemybnlNElKT9zYoNAkQOEtzp6S5ly9DnABpZt9tUcKUxSL8U2v9YBnrY4+0FlMQ+QfWlc1hm85xL3kZg0vTSC4clYt2dDnWs487EiscoWtF8+Lrwylr5c2vGGsZZp60FOTLdZZFi5m5EPYJ39k7/5DEd4i/vOpkl6WCHPGvBIH30+t2oSS8FPSuROnlYHYy1kMJuxWUPQAWX9x7kDuRAEqCTFFKTuI50rcHvzpS1phYCv7WQfW2eWtYyuMX7Y5ym3zDlG8m1TmOo0JjEscKcSyRmpNfOhD2A='


## Verification Phase


##### decryption

In [24]:
user_provided_password = "pavaniMandadi8"
BLOCK_SIZE = 16
unpad = lambda s: s[:-ord(s[len(s) - 1:])]
symbols = {"00": "0", "01": "1", "11": "*", "10": "*"}
symbols_reverse = {value:key for key, value in symbols.items()}

def decrypt(enc, password):
    private_key = hashlib.sha256(password.encode("utf-8")).digest()
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(enc[16:]))

decrypted = decrypt(encoded_string, user_provided_password)
decrypted_string = bytes.decode(decrypted)
print("AES decrypted String :", decrypted_string)
print(n_permutated_negate)

AES decrypted String : 1*01*01*1*01***001**001*1**1*1*000000001*1*1***1*1*001***1*******1******1*1*01**0001*****1**1*1**01*1***1****001**1******001*1***1***01*1*1*01***1*1*1**1*01*01*1**1**001*01*01****01**********1**1**1********01**1*00001***01*01*001*1*1***0001****1**1*1****0
0100100101001110001100010110101000000000101011101010001110111111101111110101001100001111101101011001011101111000110111111000101110111001010100111010101101001001011011000100100111100111111111101101101111111100110100000111001001000101011100001111011010111100


##### reverse NDB generation

In [25]:
def backtrack_ndb(decrypted_str):
    out = []
    for ind, each in enumerate(decrypted_str):
        if ind == 0:
            start_ind_map = {"0": ("0", "0"), "1" : ("0", "1"), "*": ("1", "*")}
            val, last_but = start_ind_map[each]
        else:
            if each == "0" and last_but == "*":
                val, last_but = ("0", "0")
            elif each == "*" and last_but == "*":
                val, last_but = ("1", "*")
            elif each == "1" and last_but == "*":
                val, last_but = ("0", "1")
            elif each == "1" and last_but == "0":
                val, last_but = ("0", "1")
            elif each == "*" and last_but == "1":
                val, last_but = ("1", "*")
            elif each == "0" and last_but == "1":
                val, last_but = ("1", "0")
            elif each == "0" and last_but == "0":
                val, last_but = ("0", "0")
            elif each == "1" and last_but == "1":
                val, last_but = ("1", "1")
            elif each == "*" and last_but == "0":
                val, last_but = ("0", "1")
        out.append(val)
    out.append(last_but)
    return "".join(out)

back_track_ndb =  backtrack_ndb(decrypted_string)

if not decrypted_string.endswith("*"):
    back_track_ndb = negate(back_track_ndb)
print(n_permutated_str)

1011011010110001110011101001010111111111010100010101110001000000010000001010110011110000010010100110100010000111001000000111010001000110101011000101010010110110100100111011011000011000000000010010010000000011001011111000110110111010100011110000100101000011


##### Inverse permutated string

In [26]:
inverse_permutated_string = npermute(back_track_ndb)
inverse_permutated_string 

'1100001010010000111100010101110110110001111101001100000000100100100000000001100001101101110010010110110100101010001101010110001000101110000001001110000100010110010100100000111100110101000000100000001000111010100010101111111110101001011100111000110101101101'

###### Binary string to hex digit 

In [59]:
divisons = [inverse_permutated_string[each: each + 4] for each in range(0, len(inverse_permutated_string), 4)]
divisons
res = []
for each in divisons:
    sum_val = sum([8*int(each[0]), 4*int(each[1]), 2*int(each[2]), 1*int(each[3])])
    res.append(sum_val)
letter_map = {10: "a", 11: "b", 12: "c", 13: "d", 14: "e", 15: "f"}

hex_digit_decrypted = "".join([letter_map[every] if every in letter_map else str(every) for every in res])
hex_digit_decrypted

'c290f15db1f4c02480186dc96d2a35622e04e116520f3502023a8affa9738d6d'

In [60]:
hex_digit == hex_digit_decrypted

True

#### Proof that n! th  permutation is the reverse of the string 

In [62]:
from itertools import permutations

sample_str = "pavani"

reverse_str = sample_str[::-1] # "inavap"

def get_last_permutations(input_str):
    perm = ["".join(x) for x in permutations(input_str)]
    return perm[-1]
    
last_perm = get_last_permutations(sample_str)
print("last permutation :", last_perm)
print("reverse string :", reverse_str)

last_perm == reverse_str

last permutation : inavap
reverse string : inavap


True

In [64]:
sample_2 = "1025689"
reverse_st = sample_2[::-1] # "9865201"

last_perm_2 = get_last_permutations(sample_2)
print("last permutation :", last_perm_2)
print("reverse string :", reverse_st)

last_perm_2 == reverse_st

last permutation : 9865201
reverse string : 9865201


True